# SPRINT - 3

## Item - Desenvolvimento do processamento dos dados para geração dos indicadores no Pipeline, e gravação na "Consumer Data"

## Item - Desenvolvimento do processamento dos dados para geração da consolidação de dados da produção no Pipeline, e gravação na "Consumer Data"

<br>
<br>

Autor.: Sérgio C. Medina

#### Declaração dos Pacotes, Libs ou Classes utilizadas no processo.

In [1]:
# Declaração dos Pacotes, Libs ou Classes utilizadas no processo.
import os
import io
import math
import pandas as pd
import gcsfs
import pyarrow
import pyarrow.parquet as pq
from google.cloud import storage
from datetime import datetime, timedelta

import sys
sys.path.append('../../pods')

# configurando variavel de ambiente com o arquivo de credenciais para conexão GCP
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "..\..\secrets\edc-igti-smedina-4920e12ac565.json"
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/sergiomedina/Downloads/edc-igti-smedina-4920e12ac565.json"

# Funções de integração com o Cloud Storage
from utilGCS import utilGCS
gcs = utilGCS(bucketName='edc-pa-i4-data')

# Funções diversas de manipulação de dados
from utilFuncs import utilFuncs
func = utilFuncs()


#### Dados na "processing-zone" pasta "DATAOP"

In [ ]:
dtexec = "2021-11-08"
source = "dataop"
folder = f"edc-pa-i4-data/processing-zone/{source}"

df_dataop = gcs.read_parquet_to_pandas(path=f"edc-pa-i4-data/processing-zone/{source}")
#df_dataop = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec)])




In [ ]:
df_dataop

#### Dados na "raw-data-zone" pasta "DATACONFIRM"

In [ ]:
dtexec = "2021-11-08"
source = "dataconfirm"
folder = f"raw-data-zone/{source}"

#df_dataconfirm = gcs.read_parquet_to_pandas(path=f"edc-pa-i4-data/processing-zone/{source}")
df_dataconfirm = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec)])


In [ ]:
df_dataconfirm

In [ ]:
df_dataconfirm.groupby(
    ['DTPROD', 'IDTURNO', 'LOTE']
).agg(
    {
        'PACKID': 'count',
        'UNIDADES': 'sum',
        'KGPACK': 'sum',
        'KGUNMED': 'mean'
    }
)


# df = pd.DataFrame(df_dataconfirm.groupby(
#     ['DTPROD', 'IDTURNO', 'LOTE']
# ).agg(
#     {
#         'PACKID': 'count',
#         'UNIDADES': 'sum',
#         'KGPACK': 'sum',
#         'KGUNMED': 'mean'
#     }
# )).reset_index()

# df

#### Dados na "raw-data-zone" pasta "DATAPROD"

In [2]:
dtexec = "2021-11-08"
lineid = "101"
source = "dataprod"
folder = f"edc-pa-i4-data/processing-zone/{source}"

#df_dataprod = gcs.read_parquet_to_pandas(path=folder)
df_dataprod = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec)])

df_dataprod['DTPROD'] = df_dataprod['DTPROD'].astype(str)


In [ ]:
df_dataprod.to_csv(path_or_buf='/Users/ych885/Downloads/2021-11-08DL.csv', sep=';', index=False )

In [ ]:
#df_dataprod = df_dataprod.loc[(df_dataprod.DTPROD==dtexec) & (df_dataprod.LINE==lineid)]
df_dataprod

In [ ]:
df_dataprod.dtypes

In [70]:
#df.groupby(['col1', 'col2']).agg({'col3':'sum','col4':'sum'})
df_sts = pd.DataFrame(df_dataprod.groupby(
    ['DTPROD', 'IDTURNO', 'LINE', 'OP', 'BATCH', 'STSID', 'STSDS']
).agg(
    {
        'TOTMIN': 'sum',
        'PC': 'sum',
        'GOOD': 'sum',
        'REJECT': 'sum'
    }
)).reset_index()


df = df_dataprod.copy()

df['TMIN'] = df['TIMESTAMP']
df['TMAX'] = df['TIMESTAMP']
df.drop(['TIMESTAMP'], axis=1, inplace=True)

df = pd.DataFrame(df.groupby(
    ['DTPROD', 'IDTURNO', 'LINE', 'OP', 'BATCH']
).agg(
    {
        'TOTMIN': 'sum',
        'PC': 'sum',
        'GOOD': 'sum',
        'REJECT': 'sum',
        'TMIN': 'min',
        'TMAX': 'max'
    }
)).reset_index()


df['TMAXLIMIT'] = df.apply(lambda row:func.end_time(row['DTPROD'], row['IDTURNO']),axis=1)
df['TMAXLIMIT'] = pd.to_datetime(df['TMAXLIMIT'])
df['TMAXDIFF'] = (df['TMAXLIMIT']-df['TMAX']).dt.seconds/60
df['WIP'] = df.apply(lambda row:func.round_down(row['TMAXDIFF'], 0),axis=1)

df['TMINLIMIT'] = df.apply(lambda row:func.start_time(row['DTPROD'], row['IDTURNO']),axis=1)
df['TMINLIMIT'] = pd.to_datetime(df['TMINLIMIT'])
df['TMINDIFF'] = (df['TMIN']-df['TMINLIMIT']).dt.seconds/60

df['DIFF'] = df['TMAXDIFF'].diff().fillna(0)

df['TOTMINADJUSTED'] = round(df['TOTMIN'] + df['DIFF'],0)

df.drop(['TMAX', 'TMAXLIMIT', 'TMAXDIFF'], axis=1, inplace=True)
df.drop(['TMIN', 'TMINLIMIT', 'TMINDIFF', 'DIFF'], axis=1, inplace=True)

# Tot. Tempo turno
df['TPTOTAL'] = 480

print('DEBUG -> [ANTES AJUSTE]:','TOTMINADJUSTED=',df['TOTMINADJUSTED'].sum(),'\n\r',df['TOTMINADJUSTED'])

df['TOTMINADJUSTED'] = df.apply(lambda row:(row['TOTMINADJUSTED'] if row['TOTMINADJUSTED']<480 else 480),axis=1)

print('DEBUG -> [DEPOIS AJUSTE]:','TOTMINADJUSTED=',df['TOTMINADJUSTED'].sum(),'\n\r',df['TOTMINADJUSTED'])

# Tot. Tempo = WORKING
df['TPWORKING'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==4) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

print('DEBUG -> [ANTES AJUSTE]:','TPWORKING=',df['TPWORKING'].sum(),'\n\r',df['TPWORKING'])

df['TPWORKING'] = df.apply(lambda row:(row['TPWORKING'] if row['TPWORKING']<=row['TOTMINADJUSTED'] else row['TOTMINADJUSTED']),axis=1)

print('DEBUG -> [DEPOIS AJUSTE]:','TPWORKING=',df['TPWORKING'].sum(),'\n\r',df['TPWORKING'])

# Tot. Tempo = TPSTOPPLAN
df['TPSTOPPLAN'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==3) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

# Tot. Tempo = TPNOALLOC
df['TPNOALLOC'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==0) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)    


# Tempo Programado para produzir
df['TPPROG'] = df['TPTOTAL'] - (df['TPNOALLOC']+df['TPSTOPPLAN'])


# Temp Ociosidade
df['TPIDLE'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID>=1) &
    (df_sts.STSID<3) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

df['TPIDLE'] = ((df['TPPROG']-(df['TPWORKING'] + df['TPIDLE']))+df['TPIDLE'])

# Unidades - Produção Teórica
df['PCTHEOR'] = df['TPWORKING'] * 1

# Unidades - Perda por performance
df['PCLOSS'] = df['PCTHEOR'] - df['PC']

# OEE Calc 
df['OEEDISP'] = round(df['TPWORKING'] / df['TPPROG'],3)

df['OEEPERF'] = round(df['PC'] / df['PCTHEOR'],3)

df['OEEQUAL'] = round(df['GOOD'] / df['PC'],3)

df['OEE'] = round(df['OEEDISP'] * df['OEEPERF'] * df['OEEQUAL'],3)




DEBUG -> [ANTES AJUSTE]: TOTMINADJUSTED= 1435.0 
 0    475.0
1    480.0
2    480.0
Name: TOTMINADJUSTED, dtype: float64
DEBUG -> [DEPOIS AJUSTE]: TOTMINADJUSTED= 1435.0 
 0    475.0
1    480.0
2    480.0
Name: TOTMINADJUSTED, dtype: float64
DEBUG -> [ANTES AJUSTE]: TPWORKING= 1388.23 
 0    422.47
1    479.83
2    485.93
Name: TPWORKING, dtype: float64
DEBUG -> [DEPOIS AJUSTE]: TPWORKING= 1382.3 
 0    422.47
1    479.83
2    480.00
Name: TPWORKING, dtype: float64


In [71]:
df

,DTPROD,IDTURNO,LINE,OP,BATCH,TOTMIN,PC,GOOD,REJECT,WIP,...,TPSTOPPLAN,TPNOALLOC,TPPROG,TPIDLE,PCTHEOR,PCLOSS,OEEDISP,OEEPERF,OEEQUAL,OEE
0,2021-11-08,1,101,211108L101,TB70394,474.87,444.0,394.0,50.0,5.0,...,0.0,0.0,480.0,57.53,422.47,-21.53,0.88,1.051,0.887,0.820
1,2021-11-08,2,101,211108L101,TB70394,479.83,452.0,450.0,2.0,6.0,...,0.0,0.0,480.0,0.17,479.83,27.83,1.00,0.942,0.996,0.938
2,2021-11-08,3,101,211108L101,TB70394,485.93,456.0,453.0,3.0,0.0,...,0.0,0.0,480.0,0.00,480.00,24.00,1.00,0.950,0.993,0.943


In [72]:
#df[['DTPROD','IDTURNO','LINE','OP','BATCH','TPTOTAL','TPPROG','TPWORKING','TOTMINADJUSTED','TPSTOPPLAN','TPNOALLOC','TPIDLE']]

#df[['DTPROD','IDTURNO','PC','GOOD','REJECT','WIP','PCTHEOR','PCLOSS']]
df[['OEEDISP', 'TPWORKING', 'TPPROG', 'OEEPERF', 'OEEQUAL', 'OEE']]

        

,OEEDISP,TPWORKING,TPPROG,OEEPERF,OEEQUAL,OEE
0,0.88,422.47,480.0,1.051,0.887,0.820
1,1.00,479.83,480.0,0.942,0.996,0.938
2,1.00,480.00,480.0,0.950,0.993,0.943


In [73]:

df_oee = pd.DataFrame(
    df.groupby(
        ['DTPROD']
    ).agg(
        {
            'TPTOTAL': 'sum',
            'TPPROG': 'sum',
            'TPWORKING': 'sum',
            'TPSTOPPLAN': 'sum',
            'TPNOALLOC': 'sum',
            'TPIDLE': 'sum',
            'PC': 'sum',
            'GOOD': 'sum',
            'REJECT': 'sum',
            'WIP': 'sum',
            'PCTHEOR': 'sum',
            'PCLOSS': 'sum',

        }
    )
).reset_index()

df_oee['OEEDISP'] = round(df_oee['TPWORKING']/df_oee['TPPROG'],3)

df_oee['OEEPERF'] = round(df_oee['PC'] / df_oee['PCTHEOR'],3)

df_oee['OEEQUAL'] = round(df_oee['GOOD'] / df_oee['PC'],3)

df_oee['OEE'] = round(df_oee['OEEDISP'] * df_oee['OEEPERF'] * df_oee['OEEQUAL'],3)


df_oee


,DTPROD,TPTOTAL,TPPROG,TPWORKING,TPSTOPPLAN,TPNOALLOC,TPIDLE,PC,GOOD,REJECT,WIP,PCTHEOR,PCLOSS,OEEDISP,OEEPERF,OEEQUAL,OEE
0,2021-11-08,1440,1440.0,1382.3,0.0,0.0,57.7,1352.0,1297.0,55.0,11.0,1382.3,30.3,0.96,0.978,0.959,0.9


In [74]:
# df.agg(
#     {
#         'OEEDISP': 'mean',
#         'OEEPERF': 'mean',
#         'OEEQUAL': 'mean',
#         'OEE': 'mean'
#     }
# )


df.groupby(
    ['DTPROD']
).agg(
    {
        'OEEDISP': 'mean',
        'OEEPERF': 'mean',
        'OEEQUAL': 'mean',
        'OEE': 'mean'
    }
)




,OEEDISP,OEEPERF,OEEQUAL,OEE
DTPROD,,,,
2021-11-08,0.96,0.981,0.958667,0.900333
